# 출처 : https://inuplace.tistory.com/577

# # 필요한 라이브러리

## ## 구글 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## ## 이미지 불러오기

In [2]:
from IPython.display import Image

# # 데이터 불러오기

In [3]:
import pandas as pd

# json 읽어오기
news_data = pd.read_json('/content/drive/MyDrive/Colab Notebooks/Learning/딥러닝/프랑소와 숄레 - 케라스 창시자에게 배우는 딥러닝/res/News_Category_Dataset_v2.json', lines=True)
news_data = news_data.loc[:, ["category", "headline"]]

# 카테고리 정수 인코딩
category_list = pd.factorize(news_data['category'])[1]
news_data['category'] = pd.factorize(news_data['category'])[0]

# 정규표현식 사용
news_data['headline'] = news_data['headline'].str.replace("[^\w]", " ")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == '':


## ## 피처 데이터

In [5]:
news_data

,category,headline
0,0,There Were 2 Mass Shootings In Texas Last Week...
1,1,Will Smith Joins Diplo And Nicky Jam For The 2...
2,1,Hugh Grant Marries For The First Time At Age 57
3,1,Jim Carrey Blasts Castrato Adam Schiff And D...
4,1,Julianna Margulies Uses Donald Trump Poop Bags...
...,...,...
200848,14,RIM CEO Thorsten Heins Significant Plans Fo...
200849,10,Maria Sharapova Stunned By Victoria Azarenka I...
200850,10,Giants Over Patriots Jets Over Colts Among M...
200851,10,Aldon Smith Arrested 49ers Linebacker Busted ...


## ## 레이블 데이터

In [6]:
category_list

Index(['CRIME', 'ENTERTAINMENT', 'WORLD NEWS', 'IMPACT', 'POLITICS',
       'WEIRD NEWS', 'BLACK VOICES', 'WOMEN', 'COMEDY', 'QUEER VOICES',
       'SPORTS', 'BUSINESS', 'TRAVEL', 'MEDIA', 'TECH', 'RELIGION', 'SCIENCE',
       'LATINO VOICES', 'EDUCATION', 'COLLEGE', 'PARENTS', 'ARTS & CULTURE',
       'STYLE', 'GREEN', 'TASTE', 'HEALTHY LIVING', 'THE WORLDPOST',
       'GOOD NEWS', 'WORLDPOST', 'FIFTY', 'ARTS', 'WELLNESS', 'PARENTING',
       'HOME & LIVING', 'STYLE & BEAUTY', 'DIVORCE', 'WEDDINGS',
       'FOOD & DRINK', 'MONEY', 'ENVIRONMENT', 'CULTURE & ARTS'],
      dtype='object')

# # 학습, 테스트 데이터 생성 및 결과데이터(category) 원핫벡터화

In [4]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# split하면서 shuffle 적용
news_train, news_test, y_train, y_test = train_test_split(news_data['headline'], news_data['category'], test_size=0.2, shuffle=True, random_state=23)

# 원핫벡터로 만들어줍시다! (num_classes로 카테고리 수 명시 가능)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# # 입력데이터(headline) 토큰화

In [7]:
# 토큰화 진행
stopwords = ['a', 'an']

X_train = []
for stc in news_train:
    token = []
    words = stc.split()
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_train.append(token)

X_test = []
for stc in news_test:
    token = []
    words = stc.split()
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_test.append(token)

# # 입력데이터 인덱싱
  * 빈도수별로 25000개의 단어를 인덱싱화한다. (데이터를 살펴보고 일정 빈도수 이상의 단어만 인덱싱되도록 적절한 숫자를 설정해준 것이다.)
  * X_train만 fit하여 이를 기준으로 학습 데이터와 테스트 데이터를 인덱싱화한다.

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(25000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# # 입력 데이터 패딩
  * headline은 모두 단어갯수가 달라 학습에 어려움이 있다.
  * 따라서 그를 맞춰준다. max_len보다 더 긴 문장은 잘려지고, 더 짧은 문장은 데이터를 0으로 채운다.

In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 15
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

# # 모델 생성 및 학습

In [18]:
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
import tensorflow

model = Sequential()
model.add(Embedding(25000, 128))
model.add(LSTM(128))
model.add(Dense(41, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['acc'])

model.fit(X_train, y_train, 
          validation_data=(X_test, y_test),
          batch_size=64,
          epochs=1)

2511/2511 [==============================] - 205s 81ms/step - loss: 2.0020 - acc: 0.4729 - val_loss: 1.5808 - val_acc: 0.5696


In [19]:
sentence = input()
token_stc = sentence.split()
encode_stc = tokenizer.texts_to_sequences([token_stc])
pad_stc = pad_sequences(encode_stc, maxlen=15)

score = model.predict(pad_stc)
print(category_list[score.argmax()], score[0, score.argmax()])

There Were 2 Mass Shootings
POLITICS 0.18300238
